In [ ]:
import random
import math
import csv

total = int(input("얼마를 충전하시겠습니까? "))
jackpot_num_list = []
previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:
        num = random.randint(1, 125)
        if num == 1:
                    return jackpot_num
        else:
            jackpot_num += 1

def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget # global로 해주어야 밖으로 반환됨
    jackpot_num = make_jackpot_num()
    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100 # 포아송과 기하분포의 cdf로 계산된 확률
        
        # 20% 기준으로 구간 설정하여 분포확인하기
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)  # 구간별 카운트를 0으로 초기화
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:  # 숫자가 해당 구간에 속하는지 확인
                    interval_counts[i] += 1
                    break  # 해당 구간에 숫자가 속하면 다음 숫자로 넘어감
        # jackpot_num_list의 길이가 0보다 큰 경우에만 평균을 계산합니다.
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0  # 또는 적절한 기본값
        
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)        
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else: # 이렇게도 되는구나 신기하다.
            start = -7  # 시작 범위
            end = 7     # 끝 범위
            gap = 0.5   # 간격
            value = 2   # 초기 대응값
            # -7부터 7까지 0.5 간격으로 범위 설정
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break  # 조건에 맞는 값을 찾으면 반복문 탈출
                start += gap  # 다음 범위로 업데이트
                value += 1  # 다음 값으로 업데이트
        
        #사라지는구간 찾기
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None  # 초기값 설정(없으면 오류)
        if len(jackpot_num_list) >= 6:  # 6번째 요소가 존재하는지 확인
            sixth_element = jackpot_num_list[5]  # 6번째 요소를 가져옴
            for index, (start, end) in enumerate(ranges, start=1):  # enumerate를 사용하여 각 구간의 인덱스를 얻습니다.
                if start <= sixth_element <= end:
                    forget = index
        
        print("\n")
        #print(bul_r)
        print(f"사라지는구간{forget}")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        #print(bul)
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        #print(f"이전 잭팟까지의 횟수 = {jackpot_num_list[-5:]}")
        
        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break
        
        interval_counts_num = [float(num) for num in interval_counts]

        #def make_num():
        # 9개의 숫자 입력 받음
        states = [interval_counts_num, probability, bul, forget, previous_betting]

        # 정확히 네 개의 숫자가 입력되었는지 확인
        if len(states) == 9 and all(n.isdigit() for n in states):
            # 문자열 입력을 정수로 변환
            states = [int(n) for n in states]

            # 기존에 저장된 숫자들을 확인
            existing_states = []
            try:
                with open('betting_history.csv', 'r', newline='') as file:
                    reader = csv.reader(file)
                    existing_states = [ [int(num) for num in row] for row in reader]
            except FileNotFoundError:
                pass  # 파일이 없으면 빈 리스트를 유지

            # 입력된 숫자 배열의 첫 세 요소와 기존에 존재하는 배열의 첫 세 요소가 일치하는지 확인
            for idx, existing in enumerate(existing_states):
                if existing[:8] == states[:8]:
                    print(f"입력하신 배열의 첫 세 숫자와 일치하는 배열이 존재합니다: {existing}, 네 번째 숫자는 {existing[3]}입니다.")
                    change = input("네 번째 숫자를 변경하시겠습니까? (yes/no): ").strip().lower()
                    if change == 'yes':
                        new_state = input("새로운 네 번째 숫자를 입력하세요: ").strip()
                        if new_state.isdigit():
                            existing_states[idx][8] = int(new_state)
                            # 변경된 배열을 파일에 다시 쓰기
                            with open('betting_history.csv', 'w', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerows(existing_states)
                            print(f"네 번째 숫자가 {new_state}로 변경되었습니다.")
                        else:
                            print("유효한 숫자를 입력하지 않았습니다.")
                    return  # 변경 여부에 관계없이 함수 종료

            # 일치하는 배열이 없으면 새로운 숫자 배열을 파일에 추가
            with open('betting_history.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(states)  # CSV의 한 행으로 숫자 쓰기
            print("betting_history.csv 파일에 숫자가 저장되었습니다.")
        else:
            print("정확히 네 개의 숫자를 입력해주세요.")

while True:
    slot_machine()

In [ ]:
import random
import math
import csv

total = int(input("얼마를 충전하시겠습니까? "))
jackpot_num_list = []
previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:
        num = random.randint(1, 125)
        if num == 1:
            return jackpot_num
        else:
            jackpot_num += 1

def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget
    jackpot_num = make_jackpot_num()
    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100
        
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:
                    interval_counts[i] += 1
                    break
        
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0
        
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)        
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else:
            start = -7
            end = 7
            gap = 0.5
            value = 2
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break
                start += gap
                value += 1
        
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None
        if len(jackpot_num_list) >= 6:
            sixth_element = jackpot_num_list[5]
            for index, (start, end) in enumerate(ranges, start=1):
                if start <= sixth_element <= end:
                    forget = index
        
        print("\n")
        print(f"사라지는구간{forget}")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        
        # 여기부터 csv파일
        interval_counts_num = [int(num) for num in interval_counts]
        states = interval_counts_num + [round(probability, 2), bul, forget, previous_betting]

        if all(isinstance(n, (int, float)) for n in states):
            existing_states = []
            try:
                with open('betting_history.csv', 'r', newline='') as file:
                    reader = csv.reader(file)
                    existing_states = [[float(num) for num in row] for row in reader]
            except FileNotFoundError:
                pass

            for idx, existing in enumerate(existing_states):
                if existing[:8] == states[:8]:
                    print(f"입력하신 상태와 일치하는 상태가 존재합니다: {existing}, 9번째 숫자는 {existing[8]}입니다.")
                    change = input("배팅을 변경하시겠습니까? (yes/no): ").strip().lower()
                    if change == 'yes':
                        new_betting = input("새로운 배팅을 입력하세요: ").strip()
                        if new_betting.isdigit():
                            existing_states[idx][8] = int(new_betting)
                            with open('betting_history.csv', 'w', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerows(existing_states)
                            print(f"배팅금이 {new_betting}로 변경되었습니다.")
                        else:
                            print("유효한 숫자를 입력하지 않았습니다.")
                    return

            with open('betting_history.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(states)
            print("betting_history.csv 파일에 숫자가 저장되었습니다.")
        else:
            print("정확히 네 개의 숫자를 입력해주세요.")
        
        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()

## 완성도 더 높이기 구간이 다 안찼을때도 csv파일로 상태 저장할수있도록

In [ ]:
import random
import math
import csv

total = int(input("얼마를 충전하시겠습니까? "))
jackpot_num_list = []
previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:
        num = random.randint(1, 125)
        if num == 1:
            return jackpot_num
        else:
            jackpot_num += 1

def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget
    jackpot_num = make_jackpot_num()
    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100
        
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:
                    interval_counts[i] += 1
                    break
        
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0
        
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)        
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else:
            start = -7
            end = 7
            gap = 0.5
            value = 2
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break
                start += gap
                value += 1
        
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None
        if len(jackpot_num_list) >= 6:
            sixth_element = jackpot_num_list[5]
            for index, (start, end) in enumerate(ranges, start=1):
                if start <= sixth_element <= end:
                    forget = index
        
        print("\n")
        print(f"사라지는구간{forget}")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        
        # 여기부터 csv파일
        interval_counts_num = [int(num) for num in interval_counts]
        states = interval_counts_num + [round(probability, 2), bul, forget, previous_betting]

        if all(isinstance(n, (int, float)) for n in states):
            existing_states = []
            try:
                with open('betting_history.csv', 'r', newline='') as file:
                    reader = csv.reader(file)
                    existing_states = [[float(num) for num in row] for row in reader]
            except FileNotFoundError:
                pass

            for idx, existing in enumerate(existing_states):
                if existing[:8] == states[:8]:
                    print(f"입력하신 상태와 일치하는 상태가 존재합니다: {existing}, 9번째 숫자는 {existing[8]}입니다.")
                    change = input("배팅을 변경하시겠습니까? (yes/no): ").strip().lower()
                    if change == 'yes':
                        new_betting = input("새로운 배팅을 입력하세요: ").strip()
                        if new_betting.isdigit():
                            existing_states[idx][8] = int(new_betting)
                            with open('betting_history.csv', 'w', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerows(existing_states)
                            print(f"배팅금이 {new_betting}로 변경되었습니다.")
                        else:
                            print("유효한 숫자를 입력하지 않았습니다.")
                    return

            with open('betting_history.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(states)
            print("betting_history.csv 파일에 숫자가 저장되었습니다.")
        else:
            print("정확히 네 개의 숫자를 입력해주세요.")
        
        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()

# 코드를 대대적으로 수정
- 구간리스트를 [0,0,0,0,0]으로 해놓고 여기에 구간별로 더하는 코드로 하면 된다.
## 최종 완성이다. 이것을 핸드폰으로 옮기며 이것이 최종 완성본이다.(2024.1.8)

In [ ]:
import random
import math
import csv

total = int(input("얼마를 충전하시겠습니까? "))
jackpot_num_list = []
previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:
        num = random.randint(1, 125)
        if num == 1:
            return jackpot_num
        else:
            jackpot_num += 1

def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget
    jackpot_num = make_jackpot_num()
    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100
        # 구간분포 처리하는 부분
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:
                    interval_counts[i] += 1
                    break
                    
        # 마지막 5개 요소평균 처리하는 부분
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0
        
        # 불균도 처리하는 부분
        bul_r = (sum(jackpot_num_list) + play_num) / 125 - len(jackpot_num_list)
        bul = int(bul_r)  # 소수점 아래를 제거하고 정수 부분만 남깁니다.
        
        # 사라지는구간 처리하는 부분
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = 0  # 기본값을 0으로 설정합니다.
        if len(jackpot_num_list) >= 6:
            sixth_element = jackpot_num_list[5]
            for index, (start, end) in enumerate(ranges, start=1):
                if start <= sixth_element <= end:
                    forget = index

        print("\n")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r:.2f} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]} 사라지는구간{forget}")
        
        # 여기부터 csv파일
        interval_counts_num = [int(num) for num in interval_counts]
        states = interval_counts_num + [round(probability, 2), bul, forget, previous_betting]

        if all(isinstance(n, (int, float)) for n in states):
            existing_states = []
            try:
                with open('betting_history.csv', 'r', newline='') as file:
                    reader = csv.reader(file)
                    existing_states = [[float(num) for num in row] for row in reader]
            except FileNotFoundError:
                pass

            for idx, existing in enumerate(existing_states):
                if existing[:8] == states[:8]:
                    print(f"같은 상황에서의 배팅금액은 {int(existing[8])}원 입니다")
                    #print(f"입력하신 상태와 일치하는 상태가 존재합니다: {existing}, 그때의 배팅은{int(existing[8])}입니다.")
                    change = input("배팅을 변경하시겠습니까? (yes/no): ").strip().lower()
                    if change == 'yes':
                        new_betting = input("새로운 배팅을 입력하세요: ").strip()
                        if new_betting.isdigit():
                            existing_states[idx][8] = int(new_betting)
                            with open('betting_history.csv', 'w', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerows(existing_states)
                            print(f"배팅금이 {new_betting}로 변경되었습니다.")
                        else:
                            print("유효한 숫자를 입력하지 않았습니다.")
                    #return

            with open('betting_history.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(states)
            print("betting_history.csv 파일에 숫자가 저장되었습니다.")
        else:
            print("정확히 네 개의 숫자를 입력해주세요.")
        
        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()

# 완벽한줄알았는데 배팅금수정하는 부분에서 오류가 발견되어 아래에서 코드를 손보고있는중에서 작업을 멈추었다.

In [ ]:
import random
import math
import csv

total = int(input("얼마를 충전하시겠습니까? "))
jackpot_num_list = []
previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:
        num = random.randint(1, 125)
        if num == 1:
            return jackpot_num
        else:
            jackpot_num += 1

def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget
    jackpot_num = make_jackpot_num()
    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100
        # 구간분포 처리하는 부분
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:
                    interval_counts[i] += 1
                    break
                    
        # 마지막 5개 요소평균 처리하는 부분
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0
        
        # 불균도 처리하는 부분
        bul_r = (sum(jackpot_num_list) + play_num) / 125 - len(jackpot_num_list)
        bul = int(bul_r)  # 소수점 아래를 제거하고 정수 부분만 남깁니다.
        
        # 사라지는구간 처리하는 부분
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = 0  # 기본값을 0으로 설정합니다.
        if len(jackpot_num_list) >= 6:
            sixth_element = jackpot_num_list[5]
            for index, (start, end) in enumerate(ranges, start=1):
                if start <= sixth_element <= end:
                    forget = index

        print("\n")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r:.2f} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]} 사라지는구간{forget}")
        
        # 여기부터 csv파일
        interval_counts_num = [int(num) for num in interval_counts]
        states = interval_counts_num + [round(probability, 2), bul, forget, previous_betting]

        if all(isinstance(n, (int, float)) for n in states):
            existing_states = []
            try:
                with open('betting_history.csv', 'r', newline='') as file:
                    reader = csv.reader(file)
                    existing_states = [[float(num) for num in row] for row in reader]
            except FileNotFoundError:
                pass

            for idx, existing in enumerate(existing_states):
                if existing[:8] == states[:8]:
                    print(f"같은 상황에서의 배팅금액은 {int(existing[8])}원 입니다")
                    #print(f"입력하신 상태와 일치하는 상태가 존재합니다: {existing}, 그때의 배팅은{int(existing[8])}입니다.")
                    change = input("배팅을 변경하시겠습니까? (yes/no): ").strip().lower()
                    if change == 'yes':
                        new_betting = input("새로운 배팅을 입력하세요: ").strip()
                        if new_betting.isdigit():
                            existing_states[idx][8] = int(new_betting)
                            with open('betting_history.csv', 'w', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerows(existing_states)
                            print(f"배팅금이 {new_betting}로 변경되었습니다.")
                        else:
                            print("유효한 숫자를 입력하지 않았습니다.")
                    break

            with open('betting_history.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(states)
            print("betting_history.csv 파일에 숫자가 저장되었습니다.")
        else:
            print("정확히 네 개의 숫자를 입력해주세요.")
        
        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()